# Reshaping and pivot tables

## Reshaping by pivoting DF objects: Pivot
Data is often stored in so-called "stacked" or "record" format:

In [1]:
import pandas as pd, numpy as np

In [2]:
import pandas.util.testing as tm

In [16]:
tm.N =3

def unpivot(frame):
    N, K = frame.shape
    data = {'value': frame.to_numpy().ravel('F'),
            'variable': np.asarray(frame.columns).repeat(N),
            'date': np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])

In [17]:
df = unpivot(tm.makeTimeDataFrame())

In [18]:
df

,date,variable,value
0,2000-01-03,A,0.286810
1,2000-01-04,A,1.134986
2,2000-01-05,A,0.011078
3,2000-01-03,B,1.684814
4,2000-01-04,B,0.146426
5,2000-01-05,B,0.530328
6,2000-01-03,C,1.064788
7,2000-01-04,C,-1.032260
8,2000-01-05,C,0.634965
9,2000-01-03,D,-1.082954


To select everything for variable `A` we could do:

In [19]:
df[df['variable'] == 'A']

,date,variable,value
0,2000-01-03,A,0.286810
1,2000-01-04,A,1.134986
2,2000-01-05,A,0.011078


But suppose we wish to do time series operations with the variables. A better representation would be where the `columns` are the unique variables and an `index` of dates identifies individual observations. To reshape the data into this form, we use the `DataFrame.pivot()` method (also implemented as a top level function pivot().

In [20]:
df.pivot(index= 'date', columns='variable', values= 'value')

variable,A,B,C,D
date,,,,
2000-01-03,0.286810,1.684814,1.064788,-1.082954
2000-01-04,1.134986,0.146426,-1.032260,0.748731
2000-01-05,0.011078,0.530328,0.634965,-1.015672


If the `values` argument is omitted, and the input DataFrame has more than one column of values which are not used as column or index inputs to pivot, then the resulting “pivoted” DataFrame will have hierarchical columns whose topmost level indicates the respective value column:

In [21]:
df['value2'] = df['value'] * 2

In [22]:
pivoted = df.pivot(index='date', columns='variable')
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.286810  1.684814  1.064788 -1.082954  0.573621  3.369627   
2000-01-04  1.134986  0.146426 -1.032260  0.748731  2.269971  0.292852   
2000-01-05  0.011078  0.530328  0.634965 -1.015672  0.022156  1.060656   

                                
variable           C         D  
date                            
2000-01-03  2.129576 -2.165908  
2000-01-04 -2.064520  1.497462  
2000-01-05  1.269931 -2.031344

You can then select subsets from the pivoted DataFrame.

In [23]:
pivoted['value2'] #pivot할때 value 지정하거나.... pivot한 후에 value를 선택하거나...

variable,A,B,C,D
date,,,,
2000-01-03,0.573621,3.369627,2.129576,-2.165908
2000-01-04,2.269971,0.292852,-2.064520,1.497462
2000-01-05,0.022156,1.060656,1.269931,-2.031344


Note that this returns a view on the underlying data in the case where the data are homogeneously-typed.

**Note** `pivot()` will error with a `ValueError: Index contains duplicate entries, cannot reshape` if the index/column pair is not unique. In this case, consider using `pivot_table()` which is a generalization of pivot that can handle duplicate values for one index/column pair.

---
## Reshaping by stacking and unstacking : STACK

Closely related to the `pivot()` method are the related `stack()` and `unstack()` methods available on Series and DataFrame. These methods are designed to work together with `MultiIndex` objects (see the section on hierarchical indexing). Here are essentially what these methods do:

- stack: “pivot” a level of the (possibly hierarchical) column labels, returning a DataFrame with an index with a new inner-most level of row labels.
- unstack: (inverse operation of stack) “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped DataFrame with a new inner-most level of column labels.

The clearest way to explain is by example. Let’s take a prior example data set from the hierarchical indexing section:

In [24]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]))

In [26]:
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

In [27]:
df = pd.DataFrame(np.random.randn(8,2), index = index, columns =['A', 'B'])

In [28]:
df

A         B
first second                    
bar   one     0.894855  1.020633
      two    -0.034269 -1.217693
baz   one     0.298847  0.355706
      two    -0.859251 -1.501724
foo   one    -0.938914  0.495715
      two     0.016600 -0.388467
qux   one    -0.138392 -1.708590
      two    -0.046337 -0.168459

In [29]:
df2 = df[:4]

In [30]:
df2

A         B
first second                    
bar   one     0.894855  1.020633
      two    -0.034269 -1.217693
baz   one     0.298847  0.355706
      two    -0.859251 -1.501724

The stack function “compresses” a level in the DataFrame’s columns to produce either:

- A Series, in the case of a simple column Index.
- A DataFrame, in the case of a MultiIndex in the columns.

If the columns have a MultiIndex, you can choose which level to stack. The stacked level becomes the new lowest level in a MultiIndex on the columns:

In [33]:
stacked = df2.stack()

In [34]:
stacked

first  second   
bar    one     A    0.894855
               B    1.020633
       two     A   -0.034269
               B   -1.217693
baz    one     A    0.298847
               B    0.355706
       two     A   -0.859251
               B   -1.501724
dtype: float64

With a “stacked” DataFrame or Series (having a `MultiIndex` as the index), the inverse operation of stack is `unstack`, which by default unstacks the **last level**:

In [35]:
stacked.unstack()

A         B
first second                    
bar   one     0.894855  1.020633
      two    -0.034269 -1.217693
baz   one     0.298847  0.355706
      two    -0.859251 -1.501724

In [40]:
stacked.unstack(1)

second        one       two
first                      
bar   A  0.894855 -0.034269
      B  1.020633 -1.217693
baz   A  0.298847 -0.859251
      B  0.355706 -1.501724

In [41]:
stacked.unstack(0)

first          bar       baz
second                      
one    A  0.894855  0.298847
       B  1.020633  0.355706
two    A -0.034269 -0.859251
       B -1.217693 -1.501724

If the indexes have names, you can use the level names instead of specifying the level numbers:

In [42]:
stacked.unstack('second')

second        one       two
first                      
bar   A  0.894855 -0.034269
      B  1.020633 -1.217693
baz   A  0.298847 -0.859251
      B  0.355706 -1.501724

Notice that the stack and unstack methods implicitly sort the index levels involved. Hence a call to stack and then unstack, or vice versa, will result in a sorted copy of the original DataFrame or Series:

In [43]:
index = pd.MultiIndex.from_product([[2, 1], ['a', 'b']])

In [44]:
df = pd.DataFrame(np.random.randn(4), index = index, columns=['A'])
df

A
2 a -0.855395
  b -1.497213
1 a  0.330833
  b  1.562228

In [45]:
all(df.unstack().stack() == df.sort_index())

True

The above code will raise a TypeError if the call to sort_index is removed.

### Multiple levels

You may also stack or unstack more than one level at a time by passing a list of levels, in which case the end result is as if each level in the list were processed individually.

In [49]:
columns = pd.MultiIndex.from_tuples([
   ....:     ('A', 'cat', 'long'), ('B', 'cat', 'long'),
   ....:     ('A', 'dog', 'short'), ('B', 'dog', 'short')],
   ....:     names=['exp', 'animal', 'hair_length']
   ....: )
   ....: 

In [50]:
df = pd.DataFrame(np.random.randn(4,4), columns=columns)

In [51]:
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,-0.860705,0.393232,-1.157715,0.211920
1,-0.732393,-0.354968,-2.482356,-0.708196
2,1.100373,-0.148284,0.047657,1.297294
3,1.537407,0.367394,0.164099,-1.532644


In [52]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.860705  0.393232
  dog    short       -1.157715  0.211920
1 cat    long        -0.732393 -0.354968
  dog    short       -2.482356 -0.708196
2 cat    long         1.100373 -0.148284
  dog    short        0.047657  1.297294
3 cat    long         1.537407  0.367394
  dog    short        0.164099 -1.532644

The list of levels can contain either level names or level numbers (but not a mixture of the two).

In [53]:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.860705  0.393232
  dog    short       -1.157715  0.211920
1 cat    long        -0.732393 -0.354968
  dog    short       -2.482356 -0.708196
2 cat    long         1.100373 -0.148284
  dog    short        0.047657  1.297294
3 cat    long         1.537407  0.367394
  dog    short        0.164099 -1.532644

### Missing data

These functions are intelligent about handling missing data and do not expect each subgroup within the hierarchical index to have the same set of labels. They also can handle the index being unsorted (but you can make it sorted by calling sort_index, of course). Here is a more complex example:

In [55]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
   ....:                             ('B', 'cat'), ('A', 'dog')],
   ....:                           names=['exp', 'animal'])

In [57]:
index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
   ....:                            ('one', 'two')],
   ....:                        names=['first', 'second'])

In [60]:
df = pd.DataFrame(np.random.randn(8,4), index=index, columns=columns)

In [65]:
df2 = df.iloc[[0,1,2,4,5,6,7]]

In [66]:
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.217366  0.560167 -0.010549  0.127380
      two    -0.394364  0.144107 -0.877865 -0.280052
baz   one    -2.180137 -0.057688 -0.194590  0.538440
foo   one    -0.007859  1.126225 -0.007332  0.037983
      two    -0.277441  2.047420 -0.232828  0.190956
qux   one     0.469783  0.963514 -0.338123  0.450729
      two    -0.734253 -0.617603  1.469026  1.404711

As mentioned above, stack can be called with a level argument to select which level in the columns to stack:

In [67]:
df2.stack('exp')

animal                 cat       dog
first second exp                    
bar   one    A    0.217366  0.127380
             B   -0.010549  0.560167
      two    A   -0.394364 -0.280052
             B   -0.877865  0.144107
baz   one    A   -2.180137  0.538440
             B   -0.194590 -0.057688
foo   one    A   -0.007859  0.037983
             B   -0.007332  1.126225
      two    A   -0.277441  0.190956
             B   -0.232828  2.047420
qux   one    A    0.469783  0.450729
             B   -0.338123  0.963514
      two    A   -0.734253  1.404711
             B    1.469026 -0.617603

In [68]:
df2.stack('animal')

exp                         A         B
first second animal                    
bar   one    cat     0.217366 -0.010549
             dog     0.127380  0.560167
      two    cat    -0.394364 -0.877865
             dog    -0.280052  0.144107
baz   one    cat    -2.180137 -0.194590
             dog     0.538440 -0.057688
foo   one    cat    -0.007859 -0.007332
             dog     0.037983  1.126225
      two    cat    -0.277441 -0.232828
             dog     0.190956  2.047420
qux   one    cat     0.469783 -0.338123
             dog     0.450729  0.963514
      two    cat    -0.734253  1.469026
             dog     1.404711 -0.617603

Unstacking can result in missing values if subgroups do not have the same set of labels. By default, missing values will be replaced with the default fill value for that data type, NaN for float, NaT for datetimelike, etc. For integer types, by default data will converted to float and missing values will be set to NaN.

In [69]:
df3 = df.iloc[  [0, 1, 4, 7],  [1, 2]]

In [70]:
df3

exp                  B          
animal             dog       cat
first second                    
bar   one     0.560167 -0.010549
      two     0.144107 -0.877865
foo   one     1.126225 -0.007332
qux   two    -0.617603  1.469026

In [71]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar     0.560167  0.144107 -0.010549 -0.877865
foo     1.126225       NaN -0.007332       NaN
qux          NaN -0.617603       NaN  1.469026

Alternatively, unstack takes an optional fill_value argument, for specifying the value of missing data.

In [72]:
df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar     5.601675e-01  1.441074e-01 -1.054851e-02 -8.778648e-01
foo     1.126225e+00 -1.000000e+09 -7.332224e-03 -1.000000e+09
qux    -1.000000e+09 -6.176028e-01 -1.000000e+09  1.469026e+00

### With a MultiIndex

Unstacking when the columns are a MultiIndex is also careful about doing the right thing:

In [75]:
df[:3].unstack()

exp            A                   B                                      A  \
animal       cat                 dog                 cat                dog   
second       one       two       one       two       one       two      one   
first                                                                         
bar     0.217366 -0.394364  0.560167  0.144107 -0.010549 -0.877865  0.12738   
baz    -2.180137       NaN -0.057688       NaN -0.194590       NaN  0.53844   

exp               
animal            
second       two  
first             
bar    -0.280052  
baz          NaN

In [77]:
df2.unstack(1)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
second       one       two       one       two       one       two       one   
first                                                                          
bar     0.217366 -0.394364  0.560167  0.144107 -0.010549 -0.877865  0.127380   
baz    -2.180137       NaN -0.057688       NaN -0.194590       NaN  0.538440   
foo    -0.007859 -0.277441  1.126225  2.047420 -0.007332 -0.232828  0.037983   
qux     0.469783 -0.734253  0.963514 -0.617603 -0.338123  1.469026  0.450729   

exp               
animal            
second       two  
first             
bar    -0.280052  
baz          NaN  
foo     0.190956  
qux     1.404711

## Reshaping by Melt()

The top-level melt() function and the corresponding DataFrame.melt() are useful to massage a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are “unpivoted” to the row axis, leaving just two non-identifier columns, “variable” and “value”. The names of those columns can be customized by supplying the var_name and value_name parameters.

For instance,

In [78]:
cheese = pd.DataFrame({'first': ['John', 'Mary'],
                       'last': ['Doe', 'Bo'],
                       'height': [5.5, 6.0],
                       'weight': [130, 150]})

In [79]:
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [80]:
cheese.melt(id_vars=['first','last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [81]:
cheese.melt(id_vars=['first', 'last'], var_name='quantity', value_name='value')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


Another way to transform is to use the wide_to_long() panel data convenience function. It is less flexible than melt(), but more user-friendly.

In [82]:
 dft = pd.DataFrame({"A1970": {0: "a", 1: "b", 2: "c"},
   ....:                     "A1980": {0: "d", 1: "e", 2: "f"},
   ....:                     "B1970": {0: 2.5, 1: 1.2, 2: .7},
   ....:                     "B1980": {0: 3.2, 1: 1.3, 2: .1},
   ....:                     "X": dict(zip(range(3), np.random.randn(3)))
   ....:                    })

In [83]:
dft["id"] = dft.index

In [84]:
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,-0.758728,0
1,b,e,1.2,1.3,1.538175,1
2,c,f,0.7,0.1,0.460143,2


In [85]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-0.758728,a,2.5
1,1970,1.538175,b,1.2
2,1970,0.460143,c,0.7
0,1980,-0.758728,d,3.2
1,1980,1.538175,e,1.3
2,1980,0.460143,f,0.1


---
## Combining with stats and GroupBy

It should be no shock that combining pivot / stack / unstack with GroupBy and the basic Series and DataFrame statistical functions can produce some very expressive and fast data manipulations.

In [86]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.217366  0.560167 -0.010549  0.127380
      two    -0.394364  0.144107 -0.877865 -0.280052
baz   one    -2.180137 -0.057688 -0.194590  0.538440
      two     1.935738  0.944812  0.175201 -1.632382
foo   one    -0.007859  1.126225 -0.007332  0.037983
      two    -0.277441  2.047420 -0.232828  0.190956
qux   one     0.469783  0.963514 -0.338123  0.450729
      two    -0.734253 -0.617603  1.469026  1.404711

In [97]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one     0.103409  0.343774
      two    -0.636114 -0.067972
baz   one    -1.187364  0.240376
      two     1.055469 -0.343785
foo   one    -0.007596  0.582104
      two    -0.255134  1.119188
qux   one     0.065830  0.707122
      two     0.367387  0.393554

In [98]:
#same result, another way
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one     0.103409  0.343774
      two    -0.636114 -0.067972
baz   one    -1.187364  0.240376
      two     1.055469 -0.343785
foo   one    -0.007596  0.582104
      two    -0.255134  1.119188
qux   one     0.065830  0.707122
      two     0.367387  0.393554

In [99]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,-0.043289,0.255203
two,0.026614,0.381534


In [104]:
df.mean().unstack()

animal,cat,dog
exp,,
A,-0.121396,0.104721
B,-0.002133,0.638869


---
## Pivot tables

### Adding margins

---
## Cross tabulations

### Normalization

### Adding margins

---
## Tiling

---
## Computing indicator / dummy variables

---
## Factorizing values

---
## Examples & F&Q

---
## Exploding a list-like column